<a href="https://colab.research.google.com/github/mdjamina/M1_TALA410A_NLP_PS1/blob/main/Project_NLP_S1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Semester 1**

Estimating a Language Model to Generate Wine Reviews

# 1. Estimating the Language Model

## a.

$p(w_{i}|w_{i-1},w_{i-2}) = \frac{p(w_{i},w_{i-1},w_{i-2})}{p(w_{i-1},w_{i-2})}$

## b.


In [1]:
def tokenization( text ):
  """

  """
  text =text.replace("’","'").replace(" '","'")
  pre_poncts = ['(', '[', '{','"','«']
  post_poncts = [')', ']', '}', '.', '?', ',', ';', '!', ':','"','»']
  apos = "'"
  line_tokens = []
  for w in text.split():
      #print("w=",w)
      new_tokens = []
      if len( w ) > 0:
          
          if (w[0] in pre_poncts) and len(w)>1:
              new_tokens += [ w[0],  w[1:] ]
              
          else:
              new_tokens += [ w ]

          if new_tokens[-1][-1] in post_poncts and len(w)>1:
              new_tokens.append( new_tokens[-1][-1])
              new_tokens[-2] = new_tokens[-2][0:-1]
              #print("+post_poncts|","new_token=",new_tokens)
              
          line_tokens += new_tokens
  return line_tokens


def make_trigrams(sent):
  """
  
  """

  words = tokenization(sent)
  return [(i,j,k) for (i,j,k) in zip(*[words[i:] for i in range(3)])]



In [2]:
sentence = "I love chocolate ice-cream."
make_trigrams(sentence)

[('I', 'love', 'chocolate'),
 ('love', 'chocolate', 'ice-cream'),
 ('chocolate', 'ice-cream', '.')]

## c.


In [3]:
from collections import defaultdict

def trigrams_counter(corpus_path):
  """

  """
  dict_counter= {}
  with open(corpus_path,'r') as corpus:
    for line in corpus:
    
      for ngrams in make_trigrams(line):
        key = (ngrams[:2])
        value = ngrams[2]
        if key not in dict_counter.keys():
          dict_counter[key]=defaultdict(int)
        
        dict_counter[key][value]+=1
  return dict_counter

In [32]:
!wget https://raw.githubusercontent.com/mdjamina/M1_TALA410A_NLP_PS1/main/wine2.txt?token=ASNSFW65NGSEHHEZL5B5PNTBYCVTW

--2021-12-20 16:10:59--  https://raw.githubusercontent.com/mdjamina/M1_TALA410A_NLP_PS1/main/wine2.txt?token=ASNSFW65NGSEHHEZL5B5PNTBYCVTW
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2819208 (2.7M) [text/plain]
Saving to: ‘wine2.txt?token=ASNSFW65NGSEHHEZL5B5PNTBYCVTW’

wine2.txt?token=ASN 100%[===================>]   2.69M  --.-KB/s    in 0.05s   

2021-12-20 16:11:00 (53.5 MB/s) - ‘wine2.txt?token=ASNSFW65NGSEHHEZL5B5PNTBYCVTW’ saved [2819208/2819208]



In [6]:
trigrams_counter('./test_reviews.txt')

{('BEGIN', 'NOW'): defaultdict(int, {'I': 2}),
 ('I', 'do'): defaultdict(int, {'not': 1}),
 ('I', 'like'): defaultdict(int, {'chocolate': 1}),
 ('NOW', 'I'): defaultdict(int, {'do': 1, 'like': 1}),
 ('chocolate', 'ice-cream'): defaultdict(int, {'.': 1}),
 ('chocolate', 'pudding'): defaultdict(int, {'.': 1}),
 ('do', 'not'): defaultdict(int, {'like': 1}),
 ('ice-cream', '.'): defaultdict(int, {'END': 1}),
 ('like', 'chocolate'): defaultdict(int, {'ice-cream': 1, 'pudding': 1}),
 ('not', 'like'): defaultdict(int, {'chocolate': 1}),
 ('pudding', '.'): defaultdict(int, {'END': 1})}

In [7]:
def make_conditional_probas(corpus_path):
  """

  """
  co_proba = {}
  dict_corpus = trigrams_counter(corpus_path)

  for key in dict_corpus:
    count_ab = sum([n for n in dict_corpus[key].values()])
    co_proba[key] = { c:count_abc/count_ab for c,count_abc in dict_corpus[key].items() }
    
  return co_proba

In [8]:
make_conditional_probas('./test_reviews.txt')

{('BEGIN', 'NOW'): {'I': 1.0},
 ('I', 'do'): {'not': 1.0},
 ('I', 'like'): {'chocolate': 1.0},
 ('NOW', 'I'): {'do': 0.5, 'like': 0.5},
 ('chocolate', 'ice-cream'): {'.': 1.0},
 ('chocolate', 'pudding'): {'.': 1.0},
 ('do', 'not'): {'like': 1.0},
 ('ice-cream', '.'): {'END': 1.0},
 ('like', 'chocolate'): {'ice-cream': 0.5, 'pudding': 0.5},
 ('not', 'like'): {'chocolate': 1.0},
 ('pudding', '.'): {'END': 1.0}}

## d.

## 2. Generation

In [9]:
import numpy as np

def sample_from_discrete_distrib(distrib):
  words, probas = zip(*distrib.items())
  probas = np.asarray(probas).astype('float64')/np.sum(probas)
  return np.random.choice(words, p=probas)

## 1. initialize the history

In [34]:
co_probas = make_conditional_probas('./wine2.txt')

sample_from_discrete_distrib(co_probas[('BEGIN', 'NOW')])

'A'

## 2. 


In [42]:
def generate(w_i1,w_i2,conditional_probas):
  """
  """

  w_i = sample_from_discrete_distrib(conditional_probas[(w_i1, w_i2)])
  print(w_i)

  if w_i=='END' : return ['']

  return [w_i].append( generate(w_i2,w_i,conditional_probas))




  

In [43]:
print(generate('BEGIN','NOW',co_probas))

A
full
,
persistent
finish
.
Great
length
.
Cabernet
Sauvignon
and
Petit
Verdot
,
Pinot
Grigio
has
good
cut
and
fine
.
Sangiovese
and
Merlot
.
Best
after
2007
.
20,000
cases
made
.
END
None


In [37]:


w_i1 = 'BEGIN'
w_i2 = 'NOW'
distrib = co_probas[(w_i1, w_i2)]
w_i = sample_from_discrete_distrib(distrib)
generated_sentence=[w_i]
while(w_i!='END'):
  w_i1 = w_i2
  w_i2 = w_i
  distrib = co_probas[(w_i1, w_i2)]
  w_i = sample_from_discrete_distrib(distrib)
  if w_i!='END' : generated_sentence.append(w_i)


generated_sentence



['Displays',
 'nice',
 'spice',
 ',',
 'with',
 'layers',
 'of',
 'ripe',
 'pear',
 ',',
 'green',
 'mango',
 'and',
 'a',
 'thick',
 'texture',
 'and',
 'pretty',
 'toast']